# Day 22

## Part 1

Enough sand has fallen; it can finally filter water for Snow Island.

Well, almost.

The sand has been falling as large compacted bricks of sand, piling up to form an impressive stack here near the edge of Island Island. In order to make use of the sand to filter water, some of the bricks will need to be broken apart - nay, disintegrated - back into freely flowing sand.

The stack is tall enough that you'll have to be careful about choosing which bricks to disintegrate; if you disintegrate the wrong brick, large portions of the stack could topple, which sounds pretty dangerous.

The Elves responsible for water filtering operations took a snapshot of the bricks while they were still falling (your puzzle input) which should let you work out which bricks are safe to disintegrate. For example:

```
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
```
Each line of text in the snapshot represents the position of a single brick at the time the snapshot was taken. The position is given as two x,y,z coordinates - one for each end of the brick - separated by a tilde (~). Each brick is made up of a single straight line of cubes, and the Elves were even careful to choose a time for the snapshot that had all of the free-falling bricks at integer positions above the ground, so the whole snapshot is aligned to a three-dimensional cube grid.

A line like 2,2,2~2,2,2 means that both ends of the brick are at the same coordinate - in other words, that the brick is a single cube.

Lines like 0,0,10~1,0,10 or 0,0,10~0,1,10 both represent bricks that are two cubes in volume, both oriented horizontally. The first brick extends in the x direction, while the second brick extends in the y direction.

A line like 0,0,1~0,0,10 represents a ten-cube brick which is oriented vertically. One end of the brick is the cube located at 0,0,1, while the other end of the brick is located directly above it at 0,0,10.

The ground is at z=0 and is perfectly flat; the lowest z value a brick can have is therefore 1. So, 5,5,1~5,6,1 and 0,2,1~0,2,5 are both resting on the ground, but 3,3,2~3,3,3 was above the ground at the time of the snapshot.

Because the snapshot was taken while the bricks were still falling, some bricks will still be in the air; you'll need to start by figuring out where they will end up. Bricks are magically stabilized, so they never rotate, even in weird situations like where a long horizontal brick is only supported on one end. Two bricks cannot occupy the same position, so a falling brick will come to rest upon the first other brick it encounters.

Here is the same example again, this time with each brick given a letter so it can be marked in diagrams:

```
1,0,1~1,2,1   <- A
0,0,2~2,0,2   <- B
0,2,3~2,2,3   <- C
0,0,4~0,2,4   <- D
2,0,5~2,2,5   <- E
0,1,6~2,1,6   <- F
1,1,8~1,1,9   <- G
```
At the time of the snapshot, from the side so the x axis goes left to right, these bricks are arranged like this:

```
 x
012
.G. 9
.G. 8
... 7
FFF 6
..E 5 z
D.. 4
CCC 3
BBB 2
.A. 1
--- 0
```
Rotating the perspective 90 degrees so the y axis now goes left to right, the same bricks are arranged like this:

```
 y
012
.G. 9
.G. 8
... 7
.F. 6
EEE 5 z
DDD 4
..C 3
B.. 2
AAA 1
--- 0
```
Once all of the bricks fall downward as far as they can go, the stack looks like this, where ? means bricks are hidden behind other bricks at that location:

```
 x
012
.G. 6
.G. 5
FFF 4
D.E 3 z
??? 2
.A. 1
--- 0
```
Again from the side:

```
 y
012
.G. 6
.G. 5
.F. 4
??? 3 z
B.C 2
AAA 1
--- 0
```
Now that all of the bricks have settled, it becomes easier to tell which bricks are supporting which other bricks:

- rick A is the only brick supporting bricks B and C.
- Brick B is one of two bricks supporting brick D and brick E.
- Brick C is the other brick supporting brick D and brick E.
- Brick D supports brick F.
- Brick E also supports brick F.
- Brick F supports brick G.
- Brick G isn't supporting any bricks.
Your first task is to figure out which bricks are safe to disintegrate. A brick can be safely disintegrated if, after removing it, no other bricks would fall further directly downward. Don't actually disintegrate any bricks - just determine what would happen if, for each brick, only that brick were disintegrated. Bricks can be disintegrated even if they're completely surrounded by other bricks; you can squeeze between bricks if you need to.

In this example, the bricks can be disintegrated as follows:

- Brick A cannot be disintegrated safely; if it were disintegrated, bricks B and C would both fall.
- Brick B can be disintegrated; the bricks above it (D and E) would still be supported by brick C.
- Brick C can be disintegrated; the bricks above it (D and E) would still be supported by brick B.
- Brick D can be disintegrated; the brick above it (F) would still be supported by brick E.
- Brick E can be disintegrated; the brick above it (F) would still be supported by brick D.
- Brick F cannot be disintegrated; the brick above it (G) would fall.
- Brick G can be disintegrated; it does not support any other bricks.
So, in this example, 5 bricks can be safely disintegrated.

Figure how the blocks will settle based on the snapshot. Once they've settled, consider disintegrating a single brick; how many bricks could be safely chosen as the one to get disintegrated?

In [86]:
from shapely.geometry import Polygon
import plotly.graph_objects as go


def plot_bricks(bricks: list[Polygon]) -> None:
    fig = go.Figure()

    # set the axes ranges
    xmin = 0
    xmax = 0
    ymin = 0
    ymax = 0
    zmin = 0
    zmax = 0

    for i, brick in enumerate(bricks):
        X, Y, Z = zip(*list(brick.exterior.coords))

        xmin = min(xmin, min(X))
        xmax = max(xmax, max(X))
        ymin = min(ymin, min(Y))
        ymax = max(ymax, max(Y))
        zmin = min(zmin, min(Z))
        zmax = max(zmax, max(Z))

        fig.add_trace(
            go.Scatter3d(
                x=X,
                y=Y,
                z=Z,
                mode="lines",
                name=f"brick {i}",
            )
        )

    fig.update_layout(
        template="plotly_dark",
        scene=dict(
            xaxis=dict(
                range=[xmin - 1, xmax + 1],
                gridcolor="rgb(35, 35, 35)",
                nticks=0,
            ),
            yaxis=dict(
                range=[ymin - 1, ymax + 1],
                gridcolor="rgb(35, 35, 35)",
                nticks=0,
            ),
            zaxis=dict(
                range=[zmin - 1, zmax + 1],
                gridcolor="rgb(35, 35, 35)",
                nticks=0,
            ),
        ),
        width=500,
        margin=dict(r=5, l=5, b=5, t=5),
    )
    fig.show()


def get_3d_vertices(x1: int, y1: int, z1: int, x2: int, y2: int, z2: int) -> list:
    # create the six faces of a brick
    offset = 0.5
    return [
        # front face
        (x1 - offset, y1 - offset, z1 - offset),
        (x1 - offset, y1 - offset, z2 + offset),
        (x1 - offset, y2 + offset, z2 + offset),
        (x1 - offset, y2 + offset, z1 - offset),
        (x1 - offset, y1 - offset, z1 - offset),  # close the loop
        # top face
        (x1 - offset, y2 + offset, z1 - offset),
        (x1 - offset, y2 + offset, z2 + offset),
        (x2 + offset, y2 + offset, z2 + offset),
        (x2 + offset, y2 + offset, z1 - offset),
        (x1 - offset, y2 + offset, z1 - offset),  # close the loop
        # bottom face
        (x1 - offset, y1 - offset, z1 - offset),
        (x1 - offset, y1 - offset, z2 + offset),
        (x2 + offset, y1 - offset, z2 + offset),
        (x2 + offset, y1 - offset, z1 - offset),
        (x1 - offset, y1 - offset, z1 - offset),  # close the loop
        # left face
        (x1 - offset, y1 - offset, z1 - offset),
        (x1 - offset, y1 - offset, z2 + offset),
        (x1 - offset, y2 + offset, z2 + offset),
        (x1 - offset, y2 + offset, z1 - offset),
        (x1 - offset, y1 - offset, z1 - offset),  # close the loop
        # right face
        (x2 + offset, y1 - offset, z1 - offset),
        (x2 + offset, y1 - offset, z2 + offset),
        (x2 + offset, y2 + offset, z2 + offset),
        (x2 + offset, y2 + offset, z1 - offset),
        (x2 + offset, y1 - offset, z1 - offset),  # close the loop
        # back face
        (x2 + offset, y1 - offset, z1 - offset),
        (x2 + offset, y1 - offset, z2 + offset),
        (x2 + offset, y2 + offset, z2 + offset),
        (x2 + offset, y2 + offset, z1 - offset),
        (x2 + offset, y1 - offset, z1 - offset),  # close the loop
        (x1 - offset, y1 - offset, z1 - offset),  # close the loop
    ]


def make_brick(coords1: str, coords2: str) -> list:
    x1, y1, z1 = [int(x) for x in coords1.split(",")]
    x2, y2, z2 = [int(x) for x in coords2.split(",")]

    if x1 != x2:
        # grows in x direction, same y/z values
        brick_coords = get_3d_vertices(min(x1, x2), y1, z1, max(x1, x2), y1, z1)
    elif y1 != y2:
        # grows in y direction, same x/z values
        brick_coords = get_3d_vertices(x1, min(y1, y2), z1, x1, max(y1, y2), z1)
    elif z1 != z2:
        # grows in z direction, same x/y values
        brick_coords = get_3d_vertices(x1, y1, min(z1, z2), x1, y1, max(z1, z2))
    else:
        # Handle the case where the brick doesn't extend in any direction (it's a single cube)
        brick_coords = get_3d_vertices(x1, y1, z1, x2, y2, z2)

    return Polygon(brick_coords)


def make_bricks(inputs: list[str]):
    bricks = []
    for line in inputs:
        coords1, coords2 = line.split("~")
        brick = make_brick(coords1, coords2)
        bricks.append(brick)
    return bricks


test_input = """
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9""".strip().splitlines()

test_bricks = make_bricks(test_input)
plot_bricks(test_bricks)

In [80]:
sample1 = test_bricks[0]
sample2 = test_bricks[2]

plot_bricks([sample1, sample2])
sample1.intersects(sample2), sample1.intersection(sample2)

(True, <LINESTRING Z (0.5 2.5 1.5, 1.5 2.5 1.5)>)

False

In [78]:
inputs = open("../inputs/22.txt").read().strip().splitlines()
bricks = make_bricks(inputs)
print(len(bricks))
plot_bricks(bricks[:20])

1394
